In [1]:
pip install azure-storage-file-datalake azure-identity azure-storage-blob

You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os, uuid, sys
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings

def initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id):
    
    try:  
        global service_client

        credential = ClientSecretCredential(tenant_id, client_id, client_secret)

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=credential)
    
    except Exception as e:
        print(e)
        
def download_file_from_directory():
    try:
        file_system_client = service_client.get_file_system_client(file_system="mycontainer")
        directory_client = file_system_client.get_directory_client("sample")
        
        local_file = open("iris.data",'wb')

        file_client = directory_client.get_file_client("iris.data")

        download = file_client.download_file()

        downloaded_bytes = download.readall()

        local_file.write(downloaded_bytes)

        local_file.close()

    except Exception as e:
     print(e)

# Enter service principal credentials with access to storage account
storage_account_name = ""
client_id = ""
client_secret = ""
tenant_id = ""

# Initialize and download dataset
initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id)
download_file_from_directory()

In [ ]:
import pandas as pd
from sklearn import svm
from joblib import dump, load

# Read training data set
train_df = pd.read_csv("iris.data", header=None, names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
print(train_df.head())
y = train_df.pop("class").values
X = train_df.values

# Train model
clf = svm.SVC()
clf.fit(X,y)

# Save model to local disk
dump(clf, 'model.joblib')

In [ ]:
# Save model to Azure blob storage

import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

container_name = "mycontainer"
local_file_name = "model.joblib"

try:
    connect_str = ""
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)

    print("\nUploading to Azure Storage as blob:\n\t" + local_file_name)

    # Upload the created file
    with open(local_file_name, "rb") as data:
        blob_client.upload_blob(data)
    
except Exception as ex:
    print('Exception:')
    print(ex)

In [ ]:
# Run model in Seldon

# Test model
print(clf.predict(X[0:2]))
print(X[0:2])

# Test model in Seldon

